In [76]:
import cplex
from cplex.exceptions import CplexError
import sys
import pandas as pd
import numpy as np
import docplex

In [77]:
def pprint(constraints: list, names: list):
    for c in constraints:
        _c = [str(_i) + "*" + n for _i, n in zip(c[0], names) if _i != 0]
        print(c[3] + ": ( [" + " ".join(_c) + "]  " + c[1] + "  " + str(c[2]) + " )") 
        

In [78]:
xls = pd.ExcelFile('logistic.xlsx')

vehicles = [1,2,3,4,5]                                # V
customers = [1,2,3,4,5,6,7,8,9]                       # C

num_vehicles = 4
num_customers = 9
num_locations = num_customers + 1                     
num_names = (num_locations ** 2) * num_vehicles

locations = [0] + customers[:num_customers]           # C': gồm các customers và Depot suất phát + Depot kết thúc

# quy ước: 
#    + (i,j) là cạnh (edge) thuộc tập E
#    + i,j là 2 customer liền kề trên 1 route

In [79]:
my_names = ["x"+str(i)+str(j) + str(k) for k in range(1,num_vehicles + 1) for i in range(0,num_locations) for j in range(0, num_locations) ]  

print(len(my_names))
print("Names ", my_names)
def display(constraints):
    pprint(constraints, my_names)

400
Names  ['x001', 'x011', 'x021', 'x031', 'x041', 'x051', 'x061', 'x071', 'x081', 'x091', 'x101', 'x111', 'x121', 'x131', 'x141', 'x151', 'x161', 'x171', 'x181', 'x191', 'x201', 'x211', 'x221', 'x231', 'x241', 'x251', 'x261', 'x271', 'x281', 'x291', 'x301', 'x311', 'x321', 'x331', 'x341', 'x351', 'x361', 'x371', 'x381', 'x391', 'x401', 'x411', 'x421', 'x431', 'x441', 'x451', 'x461', 'x471', 'x481', 'x491', 'x501', 'x511', 'x521', 'x531', 'x541', 'x551', 'x561', 'x571', 'x581', 'x591', 'x601', 'x611', 'x621', 'x631', 'x641', 'x651', 'x661', 'x671', 'x681', 'x691', 'x701', 'x711', 'x721', 'x731', 'x741', 'x751', 'x761', 'x771', 'x781', 'x791', 'x801', 'x811', 'x821', 'x831', 'x841', 'x851', 'x861', 'x871', 'x881', 'x891', 'x901', 'x911', 'x921', 'x931', 'x941', 'x951', 'x961', 'x971', 'x981', 'x991', 'x002', 'x012', 'x022', 'x032', 'x042', 'x052', 'x062', 'x072', 'x082', 'x092', 'x102', 'x112', 'x122', 'x132', 'x142', 'x152', 'x162', 'x172', 'x182', 'x192', 'x202', 'x212', 'x222', 'x23

In [80]:
cost_matrix = pd.read_excel(xls, 'Cost Matrix')
my_obj = [int(cost_matrix.iat[i,j]) for i in range(0,num_locations ) for j in range(1,num_locations + 1)]*num_vehicles

print("Object ",my_obj)

Object  [0, 18, 30, 28, 27, 8, 18, 5, 20, 5, 18, 0, 23, 10, 4, 18, 22, 13, 23, 23, 30, 23, 0, 32, 29, 18, 10, 26, 14, 26, 28, 10, 32, 0, 2, 20, 31, 17, 29, 29, 27, 4, 29, 2, 0, 19, 28, 12, 17, 28, 8, 18, 18, 20, 19, 0, 5, 8, 6, 4, 18, 22, 10, 31, 28, 5, 0, 17, 3, 16, 5, 13, 26, 17, 12, 8, 17, 0, 13, 11, 20, 23, 14, 29, 17, 6, 3, 13, 0, 10, 5, 23, 26, 29, 28, 4, 16, 11, 10, 0, 0, 18, 30, 28, 27, 8, 18, 5, 20, 5, 18, 0, 23, 10, 4, 18, 22, 13, 23, 23, 30, 23, 0, 32, 29, 18, 10, 26, 14, 26, 28, 10, 32, 0, 2, 20, 31, 17, 29, 29, 27, 4, 29, 2, 0, 19, 28, 12, 17, 28, 8, 18, 18, 20, 19, 0, 5, 8, 6, 4, 18, 22, 10, 31, 28, 5, 0, 17, 3, 16, 5, 13, 26, 17, 12, 8, 17, 0, 13, 11, 20, 23, 14, 29, 17, 6, 3, 13, 0, 10, 5, 23, 26, 29, 28, 4, 16, 11, 10, 0, 0, 18, 30, 28, 27, 8, 18, 5, 20, 5, 18, 0, 23, 10, 4, 18, 22, 13, 23, 23, 30, 23, 0, 32, 29, 18, 10, 26, 14, 26, 28, 10, 32, 0, 2, 20, 31, 17, 29, 29, 27, 4, 29, 2, 0, 19, 28, 12, 17, 28, 8, 18, 18, 20, 19, 0, 5, 8, 6, 4, 18, 22, 10, 31, 28, 5, 0, 17,

In [81]:
capacity_matrix = pd.read_excel(xls, 'Capicity')
vehicles_capacity = [capacity_matrix.iat[i,1] for i in range(0, num_vehicles)]
vehicles_capacity

[500, 70, 145, 90]

In [82]:
demand_matrix = pd.read_excel(xls, 'Demand Matrix')
demand = {customers[i]: demand_matrix.iat[i,1] for i in range(0, num_customers)}
demand

{1: 32, 2: 45, 3: 16, 4: 15, 5: 22, 6: 50, 7: 33, 8: 11, 9: 35}

In [83]:
lower_bounds = [0]*num_names
upper_bounds = [1]*num_names

print("Lb ", lower_bounds)
print("Rb ", upper_bounds)

Lb  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [84]:
# CONSTRAINTS_2: 
# # Một cạnh bất kỳ (i->j) chỉ tổn tại trên duy nhất 1 Vehicle. Nhưng i CHỈ LÀ customers, j là locations
constraint_2 = []
_i = 0
for i in customers[:num_customers]:
    con_ = [(1 if int(name[1]) != int(name[2])  and int(name[1]) == i else 0) for name in my_names]
    if len([c for c in con_  if c != 0]) > 0:
        _i += 1
        constraint_2.append((con_,"E", 1, "C2" + str(_i)))
    
display(constraint_2)


C21: ( [1*x101 1*x121 1*x131 1*x141 1*x151 1*x161 1*x171 1*x181 1*x191 1*x102 1*x122 1*x132 1*x142 1*x152 1*x162 1*x172 1*x182 1*x192 1*x103 1*x123 1*x133 1*x143 1*x153 1*x163 1*x173 1*x183 1*x193 1*x104 1*x124 1*x134 1*x144 1*x154 1*x164 1*x174 1*x184 1*x194]  E  1 )
C22: ( [1*x201 1*x211 1*x231 1*x241 1*x251 1*x261 1*x271 1*x281 1*x291 1*x202 1*x212 1*x232 1*x242 1*x252 1*x262 1*x272 1*x282 1*x292 1*x203 1*x213 1*x233 1*x243 1*x253 1*x263 1*x273 1*x283 1*x293 1*x204 1*x214 1*x234 1*x244 1*x254 1*x264 1*x274 1*x284 1*x294]  E  1 )
C23: ( [1*x301 1*x311 1*x321 1*x341 1*x351 1*x361 1*x371 1*x381 1*x391 1*x302 1*x312 1*x322 1*x342 1*x352 1*x362 1*x372 1*x382 1*x392 1*x303 1*x313 1*x323 1*x343 1*x353 1*x363 1*x373 1*x383 1*x393 1*x304 1*x314 1*x324 1*x344 1*x354 1*x364 1*x374 1*x384 1*x394]  E  1 )
C24: ( [1*x401 1*x411 1*x421 1*x431 1*x451 1*x461 1*x471 1*x481 1*x491 1*x402 1*x412 1*x422 1*x432 1*x452 1*x462 1*x472 1*x482 1*x492 1*x403 1*x413 1*x423 1*x433 1*x453 1*x463 1*x473 1*x483 1*x

In [85]:
# CONSTRAINTS_3: 
# # Lượng hàng chở đển Customer của 1 Vehicle phải nhỏ hơn bằng Capacity. Nhưng i CHỈ LÀ customers, j là locations
constraint_3 = []
# val = lambda v, c, n1, n2, n3: 0 if n3 != v else -1*int(demand[n1]) if int(n1) != 0 and n1 != n2 else c if n1 == 0 and n2 != 0 else 0
val = lambda v, c, n1, n2, n3: 0 if n3 != v else int(demand[n1]) if int(n1) != 0 and n1 != n2 else 0
_i = 0
for veh, cap in zip(vehicles[0:num_vehicles],vehicles_capacity[0:num_vehicles]): 
    _i += 1
    constraint_3.append(([val(veh, float(cap), int(name[1]), int(name[2]), int(name[3])) for name in my_names],"L", float(cap), "C3" + str(_i)))


display(constraint_3)

C31: ( [32*x101 32*x121 32*x131 32*x141 32*x151 32*x161 32*x171 32*x181 32*x191 45*x201 45*x211 45*x231 45*x241 45*x251 45*x261 45*x271 45*x281 45*x291 16*x301 16*x311 16*x321 16*x341 16*x351 16*x361 16*x371 16*x381 16*x391 15*x401 15*x411 15*x421 15*x431 15*x451 15*x461 15*x471 15*x481 15*x491 22*x501 22*x511 22*x521 22*x531 22*x541 22*x561 22*x571 22*x581 22*x591 50*x601 50*x611 50*x621 50*x631 50*x641 50*x651 50*x671 50*x681 50*x691 33*x701 33*x711 33*x721 33*x731 33*x741 33*x751 33*x761 33*x781 33*x791 11*x801 11*x811 11*x821 11*x831 11*x841 11*x851 11*x861 11*x871 11*x891 35*x901 35*x911 35*x921 35*x931 35*x941 35*x951 35*x961 35*x971 35*x981]  L  500.0 )
C32: ( [32*x102 32*x122 32*x132 32*x142 32*x152 32*x162 32*x172 32*x182 32*x192 45*x202 45*x212 45*x232 45*x242 45*x252 45*x262 45*x272 45*x282 45*x292 16*x302 16*x312 16*x322 16*x342 16*x352 16*x362 16*x372 16*x382 16*x392 15*x402 15*x412 15*x422 15*x432 15*x452 15*x462 15*x472 15*x482 15*x492 22*x502 22*x512 22*x522 22*x532 22*

In [86]:
# CONSTRAINTS_4: 
# # Với mỗi Vehicle, khi suất phát từ Depot, nó chỉ có thể đến duy nhất 1 Customer
constraint_4 = []
_i = 0
for k in vehicles[0:num_vehicles]:
    _i += 1
    constraint_4.append(([(1 if (int(name[1]) == 0 and int(name[3]) == k) else 0) for name in my_names],"E", 1, "C4" + str(_i)))
    
display(constraint_4)

C41: ( [1*x001 1*x011 1*x021 1*x031 1*x041 1*x051 1*x061 1*x071 1*x081 1*x091]  E  1 )
C42: ( [1*x002 1*x012 1*x022 1*x032 1*x042 1*x052 1*x062 1*x072 1*x082 1*x092]  E  1 )
C43: ( [1*x003 1*x013 1*x023 1*x033 1*x043 1*x053 1*x063 1*x073 1*x083 1*x093]  E  1 )
C44: ( [1*x004 1*x014 1*x024 1*x034 1*x044 1*x054 1*x064 1*x074 1*x084 1*x094]  E  1 )


In [87]:
# CONSTRAINTS_5: 
# # Trên route của Vehicle k có cạnh (i->j) : tổng tất cả cạnh đầu 'ra' của i BẰNG tổng tất cả cạnh đầu 'vào' của j
constraint_5 = []
_i = 0
for k in vehicles[0:num_vehicles]:
    for p in customers[:num_customers]: # i
        _i += 1
        foo = lambda k,p,n: 0 if int(n[3]) != k or int(n[2]) == int(n[1]) else 1 if int(n[1]) == p else -1 if int(n[2]) == p else 0                
        constraint_5.append(([foo(k, p, name) for name in my_names],"E", 0, "C5" + str(_i)))

display(constraint_5) 


C51: ( [-1*x011 1*x101 1*x121 1*x131 1*x141 1*x151 1*x161 1*x171 1*x181 1*x191 -1*x211 -1*x311 -1*x411 -1*x511 -1*x611 -1*x711 -1*x811 -1*x911]  E  0 )
C52: ( [-1*x021 -1*x121 1*x201 1*x211 1*x231 1*x241 1*x251 1*x261 1*x271 1*x281 1*x291 -1*x321 -1*x421 -1*x521 -1*x621 -1*x721 -1*x821 -1*x921]  E  0 )
C53: ( [-1*x031 -1*x131 -1*x231 1*x301 1*x311 1*x321 1*x341 1*x351 1*x361 1*x371 1*x381 1*x391 -1*x431 -1*x531 -1*x631 -1*x731 -1*x831 -1*x931]  E  0 )
C54: ( [-1*x041 -1*x141 -1*x241 -1*x341 1*x401 1*x411 1*x421 1*x431 1*x451 1*x461 1*x471 1*x481 1*x491 -1*x541 -1*x641 -1*x741 -1*x841 -1*x941]  E  0 )
C55: ( [-1*x051 -1*x151 -1*x251 -1*x351 -1*x451 1*x501 1*x511 1*x521 1*x531 1*x541 1*x561 1*x571 1*x581 1*x591 -1*x651 -1*x751 -1*x851 -1*x951]  E  0 )
C56: ( [-1*x061 -1*x161 -1*x261 -1*x361 -1*x461 -1*x561 1*x601 1*x611 1*x621 1*x631 1*x641 1*x651 1*x671 1*x681 1*x691 -1*x761 -1*x861 -1*x961]  E  0 )
C57: ( [-1*x071 -1*x171 -1*x271 -1*x371 -1*x471 -1*x571 -1*x671 1*x701 1*x711 1*x721 1*x

In [88]:
# CONSTRAINTS_6: 
# # Route của Vehicle k phải luôn trở về Depot (n+1), tức là luôn tồn tại điểm i trên route của k sao cho nó đi đến n+1
# # trong đó n+1 là Depot kết thúc
constraint_6 = []
_i = 0
for veh in vehicles[0:num_vehicles]: 
    _i += 1
    constraint_6.append(([(1 if (int(name[3]) == veh and int(name[2]) == 0 and int(name[1]) != 0) else 0) for name in my_names],"E", 1, "C6" + str(_i)))
    
display(constraint_6)


C61: ( [1*x101 1*x201 1*x301 1*x401 1*x501 1*x601 1*x701 1*x801 1*x901]  E  1 )
C62: ( [1*x102 1*x202 1*x302 1*x402 1*x502 1*x602 1*x702 1*x802 1*x902]  E  1 )
C63: ( [1*x103 1*x203 1*x303 1*x403 1*x503 1*x603 1*x703 1*x803 1*x903]  E  1 )
C64: ( [1*x104 1*x204 1*x304 1*x404 1*x504 1*x604 1*x704 1*x804 1*x904]  E  1 )


In [89]:
# CONSTRAINTS_X: vehicles ko đc đứng tại chỗ
constraint_x = []
_i = 0

_i += 1
constraint_x.append(([(1 if int(name[1]) == int(name[2]) else 0) for name in my_names],"E", 0, "CX" + str(_i)))

display(constraint_x)


CX1: ( [1*x001 1*x111 1*x221 1*x331 1*x441 1*x551 1*x661 1*x771 1*x881 1*x991 1*x002 1*x112 1*x222 1*x332 1*x442 1*x552 1*x662 1*x772 1*x882 1*x992 1*x003 1*x113 1*x223 1*x333 1*x443 1*x553 1*x663 1*x773 1*x883 1*x993 1*x004 1*x114 1*x224 1*x334 1*x444 1*x554 1*x664 1*x774 1*x884 1*x994]  E  0 )


In [90]:
# CONSTRAINTS_Y: vehicles ko đc quay ngược lại
constraint_y = []
_i = 0

for i in customers[:num_customers]:
    for j in customers[:num_customers]:
        if i != j:
            _i += 1
            constraint_y.append(([(1 if ((int(name[1]) == i and j == int(name[2])) or (int(name[1]) == j and i == int(name[2]))) else 0) for name in my_names],"L", 1, "CX" + str(_i)))

display(constraint_y)


CX1: ( [1*x121 1*x211 1*x122 1*x212 1*x123 1*x213 1*x124 1*x214]  L  1 )
CX2: ( [1*x131 1*x311 1*x132 1*x312 1*x133 1*x313 1*x134 1*x314]  L  1 )
CX3: ( [1*x141 1*x411 1*x142 1*x412 1*x143 1*x413 1*x144 1*x414]  L  1 )
CX4: ( [1*x151 1*x511 1*x152 1*x512 1*x153 1*x513 1*x154 1*x514]  L  1 )
CX5: ( [1*x161 1*x611 1*x162 1*x612 1*x163 1*x613 1*x164 1*x614]  L  1 )
CX6: ( [1*x171 1*x711 1*x172 1*x712 1*x173 1*x713 1*x174 1*x714]  L  1 )
CX7: ( [1*x181 1*x811 1*x182 1*x812 1*x183 1*x813 1*x184 1*x814]  L  1 )
CX8: ( [1*x191 1*x911 1*x192 1*x912 1*x193 1*x913 1*x194 1*x914]  L  1 )
CX9: ( [1*x121 1*x211 1*x122 1*x212 1*x123 1*x213 1*x124 1*x214]  L  1 )
CX10: ( [1*x231 1*x321 1*x232 1*x322 1*x233 1*x323 1*x234 1*x324]  L  1 )
CX11: ( [1*x241 1*x421 1*x242 1*x422 1*x243 1*x423 1*x244 1*x424]  L  1 )
CX12: ( [1*x251 1*x521 1*x252 1*x522 1*x253 1*x523 1*x254 1*x524]  L  1 )
CX13: ( [1*x261 1*x621 1*x262 1*x622 1*x263 1*x623 1*x264 1*x624]  L  1 )
CX14: ( [1*x271 1*x721 1*x272 1*x722 1*x273 1*x

In [91]:
# # CONSTRAINT_Z 
constraint_z = []
_i = 0
for k in vehicles[0:num_vehicles]:
    for p in customers[:num_customers]: # i
        _i += 1
        foo = lambda k,p,n: 0 if int(n[3]) != k or int(n[2]) == int(n[1]) or int(n[1]) == 0 or int(n[2]) == 0 else 1 if int(n[1]) == p or int(n[2]) == p else 0                
        constraint_z.append(([foo(k, p, name) for name in my_names], "L", 2, "C5" + str(_i)))

# val = lambda i, n1, n2, n3: (1 if n1 == 1 or n2 == i else 0) if n3 == k and n1 != n2 and n1 != 0 and n2 != 0 else 0
# for k in vehicles[:num_vehicles]:
#     for i in customers[:num_customers]:
#         _i += 1
#         constraint_x.append(([val(i, int(name[1]), int(name[2]), int(name[3])) for name in my_names], "L", 1, "CZ" + str(_i)))

display(constraint_z)

C51: ( [1*x121 1*x131 1*x141 1*x151 1*x161 1*x171 1*x181 1*x191 1*x211 1*x311 1*x411 1*x511 1*x611 1*x711 1*x811 1*x911]  L  2 )
C52: ( [1*x121 1*x211 1*x231 1*x241 1*x251 1*x261 1*x271 1*x281 1*x291 1*x321 1*x421 1*x521 1*x621 1*x721 1*x821 1*x921]  L  2 )
C53: ( [1*x131 1*x231 1*x311 1*x321 1*x341 1*x351 1*x361 1*x371 1*x381 1*x391 1*x431 1*x531 1*x631 1*x731 1*x831 1*x931]  L  2 )
C54: ( [1*x141 1*x241 1*x341 1*x411 1*x421 1*x431 1*x451 1*x461 1*x471 1*x481 1*x491 1*x541 1*x641 1*x741 1*x841 1*x941]  L  2 )
C55: ( [1*x151 1*x251 1*x351 1*x451 1*x511 1*x521 1*x531 1*x541 1*x561 1*x571 1*x581 1*x591 1*x651 1*x751 1*x851 1*x951]  L  2 )
C56: ( [1*x161 1*x261 1*x361 1*x461 1*x561 1*x611 1*x621 1*x631 1*x641 1*x651 1*x671 1*x681 1*x691 1*x761 1*x861 1*x961]  L  2 )
C57: ( [1*x171 1*x271 1*x371 1*x471 1*x571 1*x671 1*x711 1*x721 1*x731 1*x741 1*x751 1*x761 1*x781 1*x791 1*x871 1*x971]  L  2 )
C58: ( [1*x181 1*x281 1*x381 1*x481 1*x581 1*x681 1*x781 1*x811 1*x821 1*x831 1*x841 1*x851 1*x86

In [92]:
# Merge segment constraints
constraints_group = []
constraints_group += constraint_2 
constraints_group += constraint_3 
constraints_group += constraint_4 
constraints_group += constraint_5 
constraints_group += constraint_6
constraints_group += constraint_x
constraints_group += constraint_y
constraints_group += constraint_z


# Convert for cplex
constraints_ = []
constraint_senses = []
rhs = []
constraint_names = []
for i in constraints_group:
    constraints_.append([my_names, i[0]])
    constraint_senses.append(i[1])
    rhs.append(i[2])
    constraint_names.append(i[3])
    
    
# print(constraints_)
# print(constraint_senses)
# print(rhs)

In [93]:
# Create Cplex Problems
myProblem = None

try:
    myProblem = cplex.Cplex()     
except CplexError:
    print ("exc")


myProblem.variables.add(obj = my_obj,
                      lb = lower_bounds,
                      ub = upper_bounds,
                      names = my_names)

myProblem.objective.set_sense(myProblem.objective.sense.minimize)

for i in range(0, num_names):
    myProblem.variables.set_types(i,myProblem.variables.type.binary)
    
myProblem.linear_constraints.add(lin_expr = constraints_,
                                 senses = constraint_senses,
                                 rhs = rhs,
                                 names = constraint_names)

range(0, 166)

In [94]:
# Find the solutions
try:
    myProblem.solve()
    
    print(myProblem.get_stats())
    values=myProblem.solution.get_values()
    names=myProblem.variables.get_names()
    
    for v, n in zip(values, names):
        if v != 0:
            print(n, " - ",v)
except cplex.exceptions.errors.CplexSolverError:
    print("No solution")

Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 38 rows and 58 columns.
MIP Presolve modified 52 coefficients.
Reduced MIP has 128 rows, 342 columns, and 2022 nonzeros.
Reduced MIP has 342 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (2.32 ticks)
Found incumbent of value 248.000000 after 0.02 sec. (3.98 ticks)
Probing time = 0.00 sec. (2.53 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 128 rows, 342 columns, and 2022 nonzeros.
Reduced MIP has 342 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (1.19 ticks)
Probing time = 0.00 sec. (2.53 ticks)
Clique table members: 918.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.61 ticks)

        Nodes                             

SyntaxError: invalid syntax (<ipython-input-95-6d1e9cb58315>, line 1)